In [1]:
!pip3 install torch --extra-index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117


In [2]:
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 32.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.3
    Uninstalling transformers-4.39.3:
      Successfully uninstalled transformers-4.39.3


In [3]:
!pip install scikit-learn evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00a 0:00:01


In [4]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, balanced_accuracy_score
from torch.optim import Adam
from torch.utils.data import DataLoader

import evaluate
from sklearn.model_selection import train_test_split
import copy
from datasets import Dataset
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

2024-04-21 10:06:57.092060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 10:06:57.092155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 10:06:57.207339: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average="weighted")

In [75]:
df = pd.read_pickle('/kaggle/input/50k-25-cats-final-pkl/50k_25cats_FINAL.pkl')
X = df.text
y = df.topic

topic_to_number = {topic: i for i, topic in enumerate(y.unique())}
y = y.map(topic_to_number)

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.2, stratify=y, random_state=666)

train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'label': train_labels}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'text': test_texts, 'label': test_labels}))
full_dataset = Dataset.from_pandas(pd.DataFrame({'text': X, 'label': y}))

In [76]:
# создаём токенайзер и обучаем его нашему корпусу текстов
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2', padding=True, truncation=True)

# получаем token_ids и другие нужные атрибуты для обучения
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

train_dataset = train_dataset.map(preprocess_function)
test_dataset = test_dataset.map(preprocess_function)

Map:   0%|          | 0/39530 [00:00<?, ? examples/s]

Map:   0%|          | 0/9883 [00:00<?, ? examples/s]

In [78]:
train_dataset, test_dataset

(Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 39530
 }),
 Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 9883
 }))

In [79]:
def get_hf_model(model_id):
    model = BertForSequenceClassification.from_pretrained(model_id, num_labels=25)
    return model

def get_frozen_backbone(model):
    frozen_model = copy.deepcopy(model)

    for param in frozen_model.parameters():
        param.requires_grad = False
    for param in frozen_model.classifier.parameters():
        param.requires_grad = True
    return frozen_model

def train_model(model_id, train_dataset, test_dataset, freeze=True):
    model = get_hf_model(model_id)

    if freeze:
        model = get_frozen_backbone(model)

    training_args = TrainingArguments(
    output_dir="./third/.",
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_total_limit=6,
    weight_decay=0.01,
    learning_rate = 9e-3,
    logging_strategy = 'epoch',
    evaluation_strategy="epoch",
    save_strategy='epoch'
    )

    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator)

    trainer.train()

    return model

rubert = train_model('cointegrated/rubert-tiny2', train_dataset, test_dataset)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1
1,0.993200,0.689664,0.780770
2,0.714400,0.614905,0.801613
3,0.669300,0.587244,0.809861
4,0.646300,0.569227,0.817567
5,0.636200,0.558798,0.815886
6,0.631800,0.546804,0.821062
7,0.618900,0.548170,0.819338
8,0.619100,0.540052,0.823771
9,0.614600,0.541788,0.823276
10,0.608100,0.535796,0.824233


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [42]:
sample = test_dataset[0]['text']
number_to_topic = {v: k for k,v in topic_to_number.items()}

In [44]:
model_checkpoint = "second/checkpoint-4944/."
model = BertForSequenceClassification.from_pretrained(model_checkpoint)

inputs = tokenizer(sample, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

In [45]:
number_to_topic[predicted_class_id]

'красота'

In [70]:
from sklearn.metrics import roc_auc_score, f1_score
import torch

model_checkpoint = "second/checkpoint-4944/."
model = BertForSequenceClassification.from_pretrained(model_checkpoint)

true_labels = []
predicted_probs = []

for example in tqdm(test_dataset):
    text = example['text']
    label = example['label']
    inputs = tokenizer(text, truncation=True, padding=True)

    input_ids = torch.tensor(inputs['input_ids']).unsqueeze(0)  # Convert to tensor and unsqueeze for batch dimension
    attention_mask = torch.tensor(inputs['attention_mask']).unsqueeze(0)  # Convert to tensor and unsqueeze for batch dimension

    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
    
    probs = torch.softmax(logits, dim=1).tolist()[0]
    true_labels.append(label)
    predicted_probs.append(probs)

100%|██████████| 9883/9883 [08:38<00:00, 19.06it/s]


In [71]:
np.array(true_labels).shape, np.array(predicted_probs).shape

((9883,), (9883, 25))

In [73]:
predicted_classes = np.argmax(predicted_probs, axis=1)

In [74]:
roc_auc = roc_auc_score(true_labels, predicted_probs, multi_class='ovr')
predicted_classes = [probs.index(max(probs)) for probs in predicted_probs]
f1_weighted = f1_score(true_labels, predicted_classes, average='weighted')

print("ROC AUC:", roc_auc)
print("F1 Weighted:", f1_weighted)

ROC AUC: 0.9893781041411808
F1 Weighted: 0.8277639901991052
